## Ancient alternatively spliced exons 

From Merkin 2012

"Ancient" is relative here - the authors compared tissue-specific splicing across several mammals (cow, rhesus, mouse, rat) with chicken as the outgroup (estimated as ~300 million years apart in evolutionary time by fossil record), and exons that were alternatively spliced in all species were deemed "ancient" alternatively spliced exons

In [1]:
import itertools
import six

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)

splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
six.print_(splicing_feature_data.shape)
splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
! mkdir -p $constitutive_feature_folder
! mkdir -p $alternative_feature_folder

In [3]:
cd /projects/ps-yeolab/obotvinnik/merkin2012/

/projects/ps-yeolab/obotvinnik/merkin2012


In [4]:
import pandas as pd

df = pd.read_excel('1228186stable4.xls', header=None)
df.head()

,0,1,2,3
0,PTPRD:ENSBTAG00000010178:8:38452684:38452989:+,PTPRD:ENSMMUG00000006952:15:68642301:68642606:+,Ptprd:ENSMUSG00000028399:4:75746351:75746657:-,Q62990_RAT:ENSRNOG00000005711:5:94111899:94112...
1,UCKL1:ENSBTAG00000015798:13:54576868:54576887:+,UCKL1:ENSMMUG00000015551:10:402713:402732:+,Uckl1:ENSMUSG00000089917:2:181305943:181305962:-,NaN
2,KANK1:ENSBTAG00000010976:8:45921894:45922136:-,KANK1:ENSMMUG00000019862:15:76468463:76468702:-,Kank1:ENSMUSG00000032702:19:25498628:25498873:+,Kank1:ENSRNOG00000016023:1:228857621:228858097:+
3,Q29RZ7_BOVIN:ENSBTAG00000014786:22:49256956:49...,Pbrm1:ENSMUSG00000042323:14:31858516:31858611:+,Pbrm1:ENSRNOG00000028227:16:6478444:6478539:+,NaN
4,FAM179B:ENSBTAG00000039117:21:55239625:55239777:+,Fam179b:ENSMUSG00000035614:12:66091928:66092077:+,Fam179b:ENSRNOG00000004415:6:86344755:86344904:+,NaN


In [5]:
import numpy as np

df = df.replace('-', np.nan)

Get macaque to human liftover

In [6]:
cd /projects/ps-yeolab/genomes/rheMac/

/projects/ps-yeolab/genomes/rheMac


In [7]:
mkdir rheMac2

mkdir: cannot create directory `rheMac2': File exists


In [8]:
cd rheMac2/

/projects/ps-yeolab/genomes/rheMac/rheMac2


In [9]:
! wget http://hgdownload.cse.ucsc.edu/goldenPath/rheMac2/liftOver/rheMac2ToHg19.over.chain.gz

--2016-02-16 15:25:31--  http://hgdownload.cse.ucsc.edu/goldenPath/rheMac2/liftOver/rheMac2ToHg19.over.chain.gz
Resolving hgdownload.cse.ucsc.edu... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34213302 (33M) [application/x-gzip]
Saving to: `rheMac2ToHg19.over.chain.gz.7'

100%[======================================>] 34,213,302  95.8M/s   in 0.3s    

2016-02-16 15:25:31 (95.8 MB/s) - `rheMac2ToHg19.over.chain.gz.7' saved [34213302/34213302]



In [10]:
rheMac_series = df[1]

In [11]:
rhemac_df = rheMac_series.str.split(':').apply(pd.Series)

In [12]:
rhemac_df = rhemac_df.dropna()
rhemac_df

,0,1,2,3,4,5
0,PTPRD,ENSMMUG00000006952,15,68642301,68642606,+
1,UCKL1,ENSMMUG00000015551,10,402713,402732,+
2,KANK1,ENSMMUG00000019862,15,76468463,76468702,-
3,Pbrm1,ENSMUSG00000042323,14,31858516,31858611,+
4,Fam179b,ENSMUSG00000035614,12,66091928,66092077,+
5,ANK3,ENSMMUG00000003589,9,77118005,77118311,+
6,LOC723356,ENSMMUG00000003302,18,27874327,27874375,+
7,EFHA1,ENSMMUG00000011252,17,2170139,2170174,-
8,SYNJ2,ENSMMUG00000007153,4,104843020,104843154,-
9,KTN1,ENSMMUG00000006650,7,118677507,118677596,+


In [13]:
rhemac_df['chrom'] = 'chr' + rhemac_df[2]
rhemac_df.head()

,0,1,2,3,4,5,chrom
0,PTPRD,ENSMMUG00000006952,15,68642301,68642606,+,chr15
1,UCKL1,ENSMMUG00000015551,10,402713,402732,+,chr10
2,KANK1,ENSMMUG00000019862,15,76468463,76468702,-,chr15
3,Pbrm1,ENSMUSG00000042323,14,31858516,31858611,+,chr14
4,Fam179b,ENSMUSG00000035614,12,66091928,66092077,+,chr12


In [14]:
rhemac_df['strand'] = '+'
rhemac_df['score'] = 1000

In [15]:
rhemac_df['name'] = rhemac_df[0] + '_' + rhemac_df[1]

In [16]:
rhemac_df.head()

,0,1,2,3,4,5,chrom,strand,score,name
0,PTPRD,ENSMMUG00000006952,15,68642301,68642606,+,chr15,+,1000,PTPRD_ENSMMUG00000006952
1,UCKL1,ENSMMUG00000015551,10,402713,402732,+,chr10,+,1000,UCKL1_ENSMMUG00000015551
2,KANK1,ENSMMUG00000019862,15,76468463,76468702,-,chr15,+,1000,KANK1_ENSMMUG00000019862
3,Pbrm1,ENSMUSG00000042323,14,31858516,31858611,+,chr14,+,1000,Pbrm1_ENSMUSG00000042323
4,Fam179b,ENSMUSG00000035614,12,66091928,66092077,+,chr12,+,1000,Fam179b_ENSMUSG00000035614


In [17]:
rhemac_bed = rhemac_df.reindex(columns=['chrom', 3, 4, 'name', 'score', 'strand'])
rhemac_bed.head()

,chrom,3,4,name,score,strand
0,chr15,68642301,68642606,PTPRD_ENSMMUG00000006952,1000,+
1,chr10,402713,402732,UCKL1_ENSMMUG00000015551,1000,+
2,chr15,76468463,76468702,KANK1_ENSMMUG00000019862,1000,+
3,chr14,31858516,31858611,Pbrm1_ENSMUSG00000042323,1000,+
4,chr12,66091928,66092077,Fam179b_ENSMUSG00000035614,1000,+


In [18]:
rhemac_bed.to_csv('/projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons.bed', 
                  sep='\t', index=False, header=False)

In [19]:
! head merkin2012_rhemac_ancient_alt_exons.bed

chr16	55027280	55043304	KIF18B_ENSMMUG00000013113	1000	+
chr14	24791110	24849594	LOC713613_ENSMMUG00000015602	1000	+
chr7	126179652	126283622	RHOJ_ENSMMUG00000008203	1000	+
chr7	92490280	92873651	PRKD1_ENSMMUG00000022550	1000	+
chr6	6532824	6570828	SRD5A1_ENSMMUG00000013186	1000	+
chr7	57472232	57536686	IREB2_ENSMMUG00000023474	1000	+
chr2	43274902	43438002	ADCY5_ENSMMUG00000002230	1000	+
chr5	74655913	74744542	KIT_ENSMMUG00000003057	1000	+
chr5	53378257	53434251	SCARB2_ENSMMUG00000020913	1000	+
chr1	144408757	144417306	C1orf55_ENSMMUG00000017148	1000	+


In [20]:
! liftover -h

/bin/sh: liftover: command not found


In [21]:
! /projects/ps-yeolab/software/bin/liftOver

liftOver - Move annotations from one assembly to another
usage:
   liftOver oldFile map.chain newFile unMapped
oldFile and newFile are in bed format by default, but can be in GFF and
maybe eventually others with the appropriate flags below.
The map.chain file has the old genome as the target and the new genome
as the query.

***********************************************************************
         assemblies of the same organism. It may not do what you want
         if you are lifting between different organisms. If there has
         been a rearrangement in one of the species, the size of the
         region being mapped may change dramatically after mapping.
***********************************************************************

options:
   -minMatch=0.N Minimum ratio of bases that must remap. Default 0.95
   -gff  File is in gff/gtf format.  Note that the gff lines are converted
         separately.  It would be good to have a separate check after this
         that the line

In [22]:
# ! gunzip -v rheMac2ToHg19.over.chain.gz

In [23]:
pwd

u'/projects/ps-yeolab/genomes/rheMac/rheMac2'

In [24]:
!touch /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped

In [25]:
!chmod ug+rwx /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped

In [26]:
! ls -lha /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped

-rwxrwxr-- 1 obotvinnik yeo-group 0 Feb 16 15:25 /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped


In [27]:
%%bash

/projects/ps-yeolab/software/bin/liftOver \
    /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons.bed \
    /projects/ps-yeolab/genomes/rheMac/rheMac2/rheMac2ToHg19.over.chain \
    /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.bed \ 
    /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped

Reading liftover chains
Mapping coordinates


In [28]:
!ls -lha /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.*

-rw-r--r-- 1 obotvinnik yeo-group 29K Feb 16 15:25 /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.bed
-rwxrwxr-- 1 obotvinnik yeo-group   0 Feb 16 15:25 /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.unmapped


## Intersect with alt exons and add to Splicing feature data

In [29]:
alt_exons_bedfile_intersect = alt_exons_bedfile.replace('.bed', '_merkin2012_ancient_alt_exons.bed')
constitutive_bedfile_intersect = constitutive_bedfile.replace('.bed', '_merkin2012_ancient_alt_exons.bed')
print alt_exons_bedfile_intersect
print constitutive_bedfile_intersect

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/constitutive/exons_merkin2012_ancient_alt_exons.bed


In [30]:
! wc -l $alt_exons_bedfile

43903 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/exons.bed


In [31]:
! wc -l $constitutive_bedfile

wc: /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/constitutive/exons.bed: No such file or directory


### Intersection

In [39]:
! bedtools intersect \
    -b /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.bed \
    -a $alt_exons_bedfile -s -f 0.9 \
    -wo > $alt_exons_bedfile_intersect

! bedtools intersect \
    -b /projects/ps-yeolab/obotvinnik/merkin2012/merkin2012_rhemac_ancient_alt_exons_hg19.bed \
    -a $constitutive_bedfile -s -f 0.9 \
    -wo > $constitutive_bedfile_intersect

Error: Unable to open file /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/constitutive/exons.bed. Exiting.


In [40]:
! wc -l $alt_exons_bedfile_intersect

217 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed


In [41]:
! wc -l $constitutive_bedfile_intersect

0 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/constitutive/exons_merkin2012_ancient_alt_exons.bed


In [42]:
import pandas as pd

df = pd.read_table(alt_exons_bedfile_intersect, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr10,3162084,3162236,exon:chr10:3160974-3161061:+@exon:chr10:316208...,1000,+,chr10,3162084,3162236,LOC722330_ENSMMUG00000004935,1000,+,152
1,chr11,45967627,45967649,exon:chr11:45970437-45970497:-@exon:chr11:4596...,1000,-,chr11,45967626,45967648,PHF21A_ENSMMUG00000002617,1000,-,21
2,chr11,45967627,45967649,exon:chr11:45970437-45970497:-@exon:chr11:4596...,1000,-,chr11,45967626,45967648,PHF21A_ENSMMUG00000002617,1000,-,21
3,chr12,32890799,32890876,exon:chr12:32890039-32890095:+@exon:chr12:3289...,1000,+,chr12,32890799,32890876,LOC723655_ENSMMUG00000013655,1000,+,77
4,chr12,98989211,98989335,exon:chr12:98987753-98987913:+@exon:chr12:9898...,1000,+,chr12,98989211,98989335,SLC25A3_ENSMMUG00000023036,1000,+,124


In [43]:
exon2_merkin2012_ancient = pd.Series(splicing_feature_data.index.isin(df[3]), index=splicing_feature_data.index)
six.print_(exon2_merkin2012_ancient.shape)
exon2_merkin2012_ancient.head()

NameError: name 'splicing_feature_data' is not defined

In [ ]:
exon2_merkin2012_ancient.to_csv('{}/exon2_merkin2012_ancient.csv'.format(alternative_feature_folder))

In [ ]:
! head $alternative_feature_folder/exon2_merkin2012_ancient.csv

## Constitutive - no ancient spliced alt exons (as expected)

Write a series of all False so there's still a column with the same name in the constitutive data but it will all be "`False`"s

In [ ]:
constitutive_df = pd.read_table(constitutive_bedfile, header=None)
constitutive_df.head()

In [ ]:
constitutive_exon_ancient = pd.Series(False, index=constitutive_df[3])
constitutive_exon_ancient.head()

In [ ]:
constitutive_exon_ancient.to_csv('{}/exon2_merkin2012_ancient.csv'.format(constitutive_feature_folder))